<a href="https://colab.research.google.com/github/CaesarLCF/SoC-Estimation-Model-Development-For-Lithium-Ion-Battery-Packs/blob/main/Unscented_Kalman_Filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import math as m
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

### **1). Prepare the Datasets**

In [ ]:
# Load the battery datasets:
dataset = "battery_data.csv"      # <-- Example
df = pd.read_csv(dataset)

# Load the OCV-SoC datasets:
dataset = "OCV-SoC_data.csv"      # <-- Example
ocv = pd.read_csv(dataset)

# Prepare result columns:
df['OCV (V)'] = np.nan
df['RC Voltage (V)'] = np.nan
df['Model Voltage (V)'] = np.nan
df['Estimated SOC (%)'] = np.nan

### **2). Set up Model Parameters**

In [ ]:
# Set up model parameters:
n_UKF = 0.911                             # Coulomb counting coefficient
Q_UKF = 5.45                              # Ah (battery capacity)
R0 = 0.01                                 # ohm (internal series resistance)
Rp = 0.001                                # ohm (dynamic resistance)
Cp = 30000.0                              # F (RC branch capacitance)
time_step = 1.0                           # seconds

# Define the Polynomial:
class Polynomial:
    def __init__(self, coeffs):
        self._coeffs = [float(c) for c in coeffs]
        self._deg = len(self._coeffs) - 1
    def __call__(self, x):
        x = float(x)
        value = 0.0
        xp = 1.0
        for c in self._coeffs:
            value += c * xp
            xp *= x
        return value
    @property
    def Deriv(self):
        # returns a Polynomial representing the derivative
        if self._deg <= 0:
            return Polynomial([0.0])
        d_coeffs = [(i + 1) * self._coeffs[i + 1] for i in range(self._deg)]
        return Polynomial(d_coeffs)

# Set up polynomial of OCV vs SoC:
degree = 20                                       # coefficient of polynomial

SoC = ocv['SOC (%)'].astype(float) / 100.0        # Convert from percentage to numeric
OCV = ocv['OCV (V)'].astype(float)

coefficients = np.polyfit(SoC, OCV, degree)       # Fit polynomial of OCV-SoC
coefficients_list = coefficients.tolist()[::-1]   # Low-order first -> [a0, a1, a2,...]

OCV_Model = Polynomial(coefficients_list)         # Create OCV-SoC model

### **3). Set up Unscented Kalman Filter**

In [ ]:
# Initial State
x = 0.855                                         # x = SoC

# Dimension
n = 1
alpha = 0.001
beta = 0.01
lambda_ = 0.1
Wm = np.hstack((lambda_ / (n + lambda_), 0.5 / (n + lambda_) * np.ones(2 * n)))
Wc = np.copy(Wm)
Wc[0] += (1 - alpha**2 + beta)

# Measurement Noise Standard Deviation
std_dev = 0.00015
var = std_dev ** 2

# State Error Covariance
P = var

# Convariance of Measurement Noise
R = var

# covariance of process noise
a = 999999
Qs = var / a

### **4). Execute Unscented Kalman Filter**

In [ ]:
# Define the Unscented Kalman Filter:
for i in range(len(df)):

    # Prediction Step:
    # Step 1: Sigma Sampling Points Determination
    pk = np.sqrt((n + lambda_) * P)
    sigma1 = x + pk  # Sigma point 1, for between 2 to n+1
    sigma2 = x - pk  # Sigma point 2, for between n+2 to 2n+1
    sigma = np.array([x] + [sigma1] * n + [sigma2] * n)  # Sigma points

    # Step 2: Predict The State (SoC)
    sigma_pred = sigma - df.at[i, 'Current (A)'] * n_UKF * (time_step / (Q_UKF * 3600))
    sxk = np.sum(Wm * sigma_pred)

    # Step 3: Predict The Covariance
    spk = np.sum(Wc * (sigma_pred - sxk)**2) + Qs

    # Update Step:
    # Step 1: Update Sigma Sampling Points
    pkr = np.sqrt((n + lambda_) * spk)
    Nsigma1 = sxk + pkr
    Nsigma2 = sxk - pkr
    Nsigma = np.array([sxk] + [Nsigma1] * n + [Nsigma2] * n)

    # Step 2: Produce the Model Voltage
    gamma = np.zeros(2 * n + 1)
    ocv = np.zeros(2 * n + 1)
    for j in range(2 * n + 1):
        gamma[j] = OCV_Model(Nsigma[j]) - R0 * df.at[i, 'Current (A)'] - df.at[i, 'Current (A)'] * Rp * (1 - m.exp(-time_step / (Rp*Cp)))
        ocv[j] = OCV_Model(Nsigma[j])

    # Step 3: Mean value of Model Voltage
    syk = np.sum(Wm * gamma)

    # Step 4: Calculate the covariance of the measurement error
    pyy = np.sum(Wc * (gamma - syk)**2) + R

    # Step 4: Calculate the covariance of the measurement and state error
    pxy = np.sum(Wc * (Nsigma - sxk) * (gamma - syk))

    # Step 5: Update the Kalman Gain
    kgs = pxy / pyy

    # Estimation Step
    df.at[i,'OCV (V)'] = np.sum(Wm * ocv)
    df.at[i,'Model Voltage (V)'] = syk
    SOC_Estimate = sxk + kgs * (df.at[i, 'Voltage (V)'] - syk)  # Update the state SoC
    if SOC_Estimate * 100 > 100:
      df.at[i, 'Estimated SOC (%)'] = 100
    elif SOC_Estimate * 100 <= 0:
      df.at[i, 'Estimated SOC (%)'] = 0
    else:
      df.at[i, 'Estimated SOC (%)'] = SOC_Estimate * 100

    # Step 6 : Update the Covariance P
    P = spk - kgs * pyy * kgs

    # Step 7 : Update the state (SoC)
    x = SOC_Estimate

### **5). Evaluate Unscented Kalman Filter**

In [ ]:
# Evaluation of OKF: SoC
print('Performance Metrics of SoC:')
# Calculate MAE
mae = mean_absolute_error(df['SOC (%)'], df['Estimated SOC (%)'])
print(f'Mean Absolute Error (MAE): {mae}')
# Calculate RMSE
mse = mean_squared_error(df['SOC (%)'], df['Estimated SOC (%)'])
rmse = (np.sqrt(mse))
print(f'Root Mean Squared Error (RMSE): {rmse}')
# Calculate R-squared
r2 = r2_score(df['SOC (%)'], df['Estimated SOC (%)'])
print(f'Coefficient of Determination (R²): {r2}')

print()

# Evaluation of EKF: Terminal Voltage
print('Performance Metrics of Terminal Voltage:')
# Calculate MAE
mae = mean_absolute_error(df['Voltage (V)'], df['Model Voltage (V)'])
print(f'Mean Absolute Error (MAE): {mae}')
# Calculate RMSE
mse = mean_squared_error(df['Voltage (V)'], df['Model Voltage (V)'])
rmse = (np.sqrt(mse))
print(f'Root Mean Squared Error (RMSE): {rmse}')
# Calculate R-squared
r2 = r2_score(df['Voltage (V)'], df['Model Voltage (V)'])
print(f'Coefficient of Determination (R²): {r2}')

In [ ]:
# Plot the data of Voltage:
plt.figure(figsize=(10, 6))
plt.plot(df['Time (s)'], df['Model Voltage (V)'], label='Model Voltage (V)', color='b', linestyle='--')
plt.plot(df['Time (s)'], df['OCV (V)'], label='OCV (V)', color='green')
plt.plot(df['Time (s)'], df['Voltage (V)'], label='Measured Voltage (V)', color='r')
plt.xlabel('Step Time (k)')
plt.ylabel('Voltage (V)')
plt.title('Voltage (V)')
plt.legend()
plt.show()

# Plot the data of SoC:
plt.figure(figsize=(10, 6))
plt.plot(df['Time (s)'], df['Estimated SOC (%)'], label='Estimated SOC (%)', color='b', linestyle='--')
plt.plot(df['Time (s)'], df['SOC (%)'], label='Actual SOC (%)', color='green')
plt.plot(df['Time (s)'], df['Predicted SOC (%)'], label='Predicted SOC (%)', color='r')
plt.xlabel('Step Time (k)')
plt.ylabel('State of Charge (%)')
plt.title('State of Charge, SoC (%)' )
plt.legend()
plt.show()

In [ ]:
# Calculate the SoC Error:
df['SOC Error'] = df['Estimated SOC (%)'] - df['SOC (%)']

# Plot the SoC Error:
plt.figure(figsize=(10, 6))
plt.plot(df['Time (s)'], df['SOC Error'], label='Prediction Error', color='m')
plt.axhline(y=0, color='k', linestyle='--')
plt.xlabel('Step Time (k)')
plt.ylabel('Error (Actual SOC - Estimated SOC)')
plt.title('Prediction Error Over Time')
plt.legend()
plt.show()

# Calculate the Voltage Error:
df['Volatge Error'] = df['Model Voltage (V)'] - df['Voltage (V)']

# Plot the errors
plt.figure(figsize=(10, 6))
plt.plot(df['Time (s)'], df['Volatge Error'], label='Prediction Error', color='m')
plt.axhline(y=0, color='k', linestyle='--')
plt.xlabel('Step Time (k)')
plt.ylabel('Error (Model Voltage - Measured Voltage)')
plt.title('Prediction Error Over Time')
plt.legend()
plt.show()

In [ ]:
# Calculate the SoC Percentage Error:
df['SOC Percentage Error'] = ((df['Estimated SOC (%)'] - df['SOC (%)']) / df['SOC (%)']) * 100

# Plot the SoC Percentage Error:
plt.figure(figsize=(10, 6))
plt.plot(df['Time (s)'], df['SOC Percentage Error'], label='SOC Percentage Error', color='m')
plt.axhline(y=0, color='k', linestyle='--')
plt.xlabel('Step Time (k)')
plt.ylabel('SOC Percentage Error')
plt.title('SOC Percentage Error Over Time')
plt.legend()
plt.show()

# Calculate the Voltage Percentage Error:
df['Volatge Percentage Error'] = ((df['Model Voltage (V)'] - df['Voltage (V)']) / df['Voltage (V)']) * 100

# Plot the errors
plt.figure(figsize=(10, 6))
plt.plot(df['Time (s)'], df['Volatge Percentage Error'], label='Voltage Percentage Error', color='m')
plt.axhline(y=0, color='k', linestyle='--')
plt.xlabel('Step Time (k)')
plt.ylabel('Voltage Percentage Error')
plt.title('Voltage Percentage Error Over Time')
plt.legend()
plt.show()